# Preprocess train dataset

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

#Math tools
from scipy import stats
from scipy.stats import skew,norm  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import scipy.stats as stats

In [2]:
def ingest_raw_data():
    path = Path()
    i = 1
    d = {}
    for file in path.glob('raw_data/*.csv'):
        d[str(i)] = pd.read_csv(file)
        print(f'Loaded df{str(i)}: {file}')
        i += 1
    return d

In [3]:
d = ingest_raw_data()

Loaded df1: raw_data\sample_submission.csv
Loaded df2: raw_data\test.csv
Loaded df3: raw_data\train.csv


In [4]:
test = d['2'][:]
df = d['3'][:]

In [5]:
df_ID = df['Id']

In [6]:
df.drop(['Id'], axis=1, inplace=True)

C:\Users\fmhiga\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Missing Data

In [7]:
#missing data
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageType,81,0.055479
GarageCond,81,0.055479
GarageFinish,81,0.055479
GarageQual,81,0.055479


In [8]:
#missing data
total = test.isnull().sum().sort_values(ascending=False)
percent = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
PoolQC,1456,0.997944
MiscFeature,1408,0.965045
Alley,1352,0.926662
Fence,1169,0.801234
FireplaceQu,730,0.500343
LotFrontage,227,0.155586
GarageCond,78,0.053461
GarageQual,78,0.053461
GarageYrBlt,78,0.053461
GarageFinish,78,0.053461


In [9]:
# PoolQC: Pool quality
df['PoolQC'].value_counts()

Gd    3
Fa    2
Ex    2
Name: PoolQC, dtype: int64

In [10]:
# MiscFeature: Miscellaneous feature not covered in other categories
df['MiscFeature'].value_counts()

Shed    49
Gar2     2
Othr     2
TenC     1
Name: MiscFeature, dtype: int64

In [11]:
# Alley: Type of alley access
df['Alley'].value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [12]:
# Fence: Fence quality
df['Fence'].value_counts()

MnPrv    157
GdPrv     59
GdWo      54
MnWw      11
Name: Fence, dtype: int64

In [13]:
# FireplaceQu: Fireplace quality
df['FireplaceQu'].value_counts()

Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: FireplaceQu, dtype: int64

In [14]:
# LotFrontage: Linear feet of street connected to property
# Neighborhood: Physical locations within Ames city limits
df['LotFrontage'].value_counts()

60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
        ... 
106.0      1
38.0       1
138.0      1
140.0      1
137.0      1
Name: LotFrontage, Length: 110, dtype: int64

In [15]:
# GarageCond: Garage condition
df['GarageCond'].value_counts()

TA    1326
Fa      35
Gd       9
Po       7
Ex       2
Name: GarageCond, dtype: int64

In [16]:
# GarageCond: Garage condition
df['GarageQual'].value_counts()

TA    1311
Fa      48
Gd      14
Po       3
Ex       3
Name: GarageQual, dtype: int64

In [17]:
# GarageYrBlt: Year garage was built
df['GarageYrBlt'].value_counts()

2005.0    65
2006.0    59
2004.0    53
2003.0    50
2007.0    49
          ..
1908.0     1
1927.0     1
1933.0     1
1900.0     1
1906.0     1
Name: GarageYrBlt, Length: 97, dtype: int64

In [18]:
# GarageFinish: Interior finish of the garage
df['GarageFinish'].value_counts()

Unf    605
RFn    422
Fin    352
Name: GarageFinish, dtype: int64

In [19]:
# GarageType: Garage location
df['GarageType'].value_counts()

Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64

In [20]:
# BsmtCond: General condition of the basement
df['BsmtCond'].value_counts()

TA    1311
Gd      65
Fa      45
Po       2
Name: BsmtCond, dtype: int64

In [21]:
# BsmtQual: Height of the basement
df['BsmtQual'].value_counts()

TA    649
Gd    618
Ex    121
Fa     35
Name: BsmtQual, dtype: int64

In [22]:
# BsmtExposure: Walkout or garden level basement walls
df['BsmtExposure'].value_counts()

No    953
Av    221
Gd    134
Mn    114
Name: BsmtExposure, dtype: int64

In [23]:
# BsmtFinType1: Quality of basement finished area
df['BsmtFinType1'].value_counts()

Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
LwQ     74
Name: BsmtFinType1, dtype: int64

In [24]:
# BsmtFinType2: Quality of second finished area (if present)
df['BsmtFinType2'].value_counts()

Unf    1256
Rec      54
LwQ      46
BLQ      33
ALQ      19
GLQ      14
Name: BsmtFinType2, dtype: int64

In [25]:
# MasVnrArea: Masonry veneer area in square feet
df['MasVnrArea'].value_counts()

0.0      861
72.0       8
180.0      8
108.0      8
120.0      7
        ... 
651.0      1
337.0      1
415.0      1
293.0      1
621.0      1
Name: MasVnrArea, Length: 327, dtype: int64

In [26]:
# Electrical: Electrical system
df['Electrical'].value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [27]:
# Electrical: Electrical system
df['MSZoning'].value_counts()

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

In [28]:
# BsmtHalfBath: Basement half bathrooms
df['BsmtHalfBath'].value_counts()

0    1378
1      80
2       2
Name: BsmtHalfBath, dtype: int64

In [29]:
print("Total No. of missing value {} before Imputation".format(sum(df.isnull().sum())))

class missing_values():
    
    for col_cat in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageCond', 'GarageQual', 'GarageYrBlt',
               'GarageFinish', 'GarageType', 'BsmtCond', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
               'MasVnrType', 'MSSubClass']:
        df[col_cat] = df[col_cat].fillna('None')
    
    # LotFrontage : Since the area of each street connected to the house property most likely have a similar area to 
    # other houses in its neighborhood , we can fill in missing values by the median LotFrontage of the neighborhood.
    #Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
    df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])
    # MSZoning (The general zoning classification) : 'RL' is by far the most common value. So we can fill in 
    # missing values with 'RL'
    df['MSZoning'] = df['MSZoning'].fillna(df['MSZoning'].mode()[0])
    # Electrical : It has one NA value. Since this feature has mostly 'SBrkr', we can set that for the missing value.
    df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])
    # Functional : data description says NA means typical
    df["Functional"] = df["Functional"].fillna("Typ")
    # KitchenQual: Only one NA value, and same as Electrical, we set 'TA' (which is the most frequent) for the missing 
    # value in KitchenQual.
    df['KitchenQual'] = df['KitchenQual'].fillna(df['KitchenQual'].mode()[0])
    # Exterior1st and Exterior2nd : Again Both Exterior 1 & 2 have only one missing value. We will just substitute in 
    # the most common string
    df['Exterior1st'] = df['Exterior1st'].fillna(df['Exterior1st'].mode()[0])
    df['Exterior2nd'] = df['Exterior2nd'].fillna(df['Exterior2nd'].mode()[0])
    # SaleType : Fill in again with most frequent which is "WD"
    df['SaleType'] = df['SaleType'].fillna(df['SaleType'].mode()[0])

    for col_num in ['MasVnrArea', 'BsmtHalfBath']:
        df[col_num] = df[col_num].fillna(0)
    
missing_values()

print("Total No. of missing value {} after Imputation".format(sum(df.isnull().sum())))

Total No. of missing value 6965 before Imputation
Total No. of missing value 0 after Imputation


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [30]:
df['KitchenQual'].isna().sum()

0

### Change number to categorial features

In [31]:
# converting some numeric features to string

#MSSubClass: The building class
df['MSSubClass'] = df['MSSubClass'].apply(str)
# YrSold: Year Sold
df['YrSold'] = df['YrSold'].astype(str)
#MoSold: Month Sold
df['MoSold'] = df['MoSold'].astype(str)
# GarageYrBlt: Year garage was built
df['GarageYrBlt'] = df['GarageYrBlt'].astype(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

# Feature Engineering

In [32]:
# YrSold: Year Sold Original construction date + YearRemodAdd: Remodel date
df['YearsSinceRemodel'] = df['YrSold'].astype(int) + df['YearRemodAdd'].astype(int)
# OverallQual: Overall material and finish quality + OverallCond: Overall condition rating
df['Total_Home_Quality'] = df['OverallQual'] + df['OverallCond']
# TotalBsmtSF: Total square feet of basement area + 1stFlrSF: First Floor square feet + 2ndFlrSF: Second floor square feet
df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
# YearBuilt: Original construction date + YearRemodAdd: Remodel date
df['YrBltAndRemod'] = df['YearBuilt'] + df['YearRemodAdd']
# BsmtFinSF1: Type 1 finished square feet + BsmtFinSF2: Type 2 finished square feet + 
# 1stFlrSF: First Floor square feet + 2ndFlrSF: Second floor square feet
df['Total_sqr_footage'] = (df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['1stFlrSF'] + df['2ndFlrSF'])
# FullBath: Full bathrooms above grade + HalfBath: Half baths above grade + 
# BsmtFullBath: Basement full bathrooms + BsmtHalfBath: Basement half bathrooms
df['Total_Bathrooms'] = (df['FullBath'] + (0.5 * df['HalfBath']) + df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath']))
# OpenPorchSF: Open porch area in square feet + 3SsnPorch: Three season porch area in square feet +
# EnclosedPorch: Enclosed porch area in square feet + ScreenPorch: Screen porch area in square feet + 
# WoodDeckSF: Wood deck area in square feet
df['Total_porch_sf'] = (df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF'])

In [33]:
# simplified features
df['haspool'] = df['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
df['has2ndfloor'] = df['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
df['hasgarage'] = df['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
df['hasbsmt'] = df['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
df['hasfireplace'] = df['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 92 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MSSubClass          1460 non-null   object 
 1   MSZoning            1460 non-null   object 
 2   LotFrontage         1460 non-null   float64
 3   LotArea             1460 non-null   int64  
 4   Street              1460 non-null   object 
 5   Alley               1460 non-null   object 
 6   LotShape            1460 non-null   object 
 7   LandContour         1460 non-null   object 
 8   Utilities           1460 non-null   object 
 9   LotConfig           1460 non-null   object 
 10  LandSlope           1460 non-null   object 
 11  Neighborhood        1460 non-null   object 
 12  Condition1          1460 non-null   object 
 13  Condition2          1460 non-null   object 
 14  BldgType            1460 non-null   object 
 15  HouseStyle          1460 non-null   object 
 16  Overal

### Skewed features

In probability theory and statistics, skewness is a measure of the asymmetry of the probability distribution of a real-valued random variable about its mean. The skewness value can be positive or negative, or undefined.

For a unimodal distribution, negative skew commonly indicates that the tail is on the left side of the distribution, and positive skew indicates that the tail is on the right. In cases where one tail is long but the other tail is fat, skewness does not obey a simple rule. For example, a zero value means that the tails on both sides of the mean balance out overall; this is the case for a symmetric distribution, but can also be true for an asymmetric distribution where one tail is long and thin, and the other is short but fat.

In [35]:
# Filter the skewed features
numeric = df.select_dtypes(include='number').columns
skew_features = df[numeric].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(10)

There are 30 numerical features with Skew > 0.5 :


MiscVal         24.451640
PoolArea        14.813135
haspool         14.337930
LotArea         12.195142
3SsnPorch       10.293752
LowQualFinSF     9.002080
KitchenAbvGr     4.483784
BsmtFinSF2       4.250888
ScreenPorch      4.117977
BsmtHalfBath     4.099186
dtype: float64

### Box Cox Transformation of (highly) skewed features

Performing Box Cox transformations is a powerful and elegant way of normalizing skewed data and can lead to significant improvements in machine learning performance. 

In [36]:
# Normalize skewed features using boxcox
for i in skew_index:
    df[i] = boxcox1p(df[i], boxcox_normmax(df[i] + 1))

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3538: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


# Getting dummy categorical features

In [100]:
# Get dummies
df = pd.get_dummies(df, prefix_sep='_', drop_first=True)

# prefix_sep='_',

In [101]:
df.shape

(1460, 394)

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 394 entries, LotFrontage to SaleCondition_Partial
dtypes: float64(31), int32(1), int64(13), uint8(349)
memory usage: 1005.3 KB


In [103]:
def remove_overfit_features(df,weight):
    overfit = []
    for i in df.columns:
        counts = df[i].value_counts()
        zeros = counts.iloc[0]
        if zeros / len(df) * 100 > weight:
            overfit.append(i)
    overfit = list(overfit)
    return overfit

overfitted_features = remove_overfit_features(df,99)

In [104]:
overfitted_features

['PoolArea',
 'haspool',
 'MSSubClass_180',
 'MSSubClass_40',
 'MSSubClass_45',
 'Street_Pave',
 'LotShape_IR3',
 'Utilities_NoSeWa',
 'LotConfig_FR3',
 'LandSlope_Sev',
 'Neighborhood_Blueste',
 'Neighborhood_NPkVill',
 'Neighborhood_Veenker',
 'Condition1_PosA',
 'Condition1_RRAe',
 'Condition1_RRNe',
 'Condition1_RRNn',
 'Condition2_Feedr',
 'Condition2_PosA',
 'Condition2_PosN',
 'Condition2_RRAe',
 'Condition2_RRAn',
 'Condition2_RRNn',
 'HouseStyle_1.5Unf',
 'HouseStyle_2.5Fin',
 'HouseStyle_2.5Unf',
 'RoofStyle_Gambrel',
 'RoofStyle_Mansard',
 'RoofStyle_Shed',
 'RoofMatl_Membran',
 'RoofMatl_Metal',
 'RoofMatl_Roll',
 'RoofMatl_Tar&Grv',
 'RoofMatl_WdShake',
 'RoofMatl_WdShngl',
 'Exterior1st_AsphShn',
 'Exterior1st_BrkComm',
 'Exterior1st_CBlock',
 'Exterior1st_ImStucc',
 'Exterior1st_Stone',
 'Exterior2nd_AsphShn',
 'Exterior2nd_Brk Cmn',
 'Exterior2nd_CBlock',
 'Exterior2nd_ImStucc',
 'Exterior2nd_Other',
 'Exterior2nd_Stone',
 'ExterQual_Fa',
 'ExterCond_Po',
 'Foundation_S

In [105]:
for c in df.columns:
    print(c)

LotFrontage
LotArea
OverallQual
OverallCond
YearBuilt
YearRemodAdd
MasVnrArea
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
TotRmsAbvGrd
Fireplaces
GarageCars
GarageArea
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MiscVal
SalePrice
YearsSinceRemodel
Total_Home_Quality
TotalSF
YrBltAndRemod
Total_sqr_footage
Total_Bathrooms
Total_porch_sf
haspool
has2ndfloor
hasgarage
hasbsmt
hasfireplace
MSSubClass_160
MSSubClass_180
MSSubClass_190
MSSubClass_20
MSSubClass_30
MSSubClass_40
MSSubClass_45
MSSubClass_50
MSSubClass_60
MSSubClass_70
MSSubClass_75
MSSubClass_80
MSSubClass_85
MSSubClass_90
MSZoning_FV
MSZoning_RH
MSZoning_RL
MSZoning_RM
Street_Pave
Alley_None
Alley_Pave
LotShape_IR2
LotShape_IR3
LotShape_Reg
LandContour_HLS
LandContour_Low
LandContour_Lvl
Utilities_NoSeWa
LotConfig_CulDSac
LotConfig_FR2
LotConfig_FR3
LotConfig_Inside
LandSlope_Mod
LandSlope_Sev


In [108]:
df.drop(overfitted_features, inplace=True, axis=1)

In [109]:
df.shape

(1460, 259)

In [110]:
df.to_csv('preprossed_data/preprossed_data.csv', index=False)